In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
from datasets import Dataset
import torch
from transformers import AutoTokenizer

# Load model in 4-bit precision mode
model_name = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    load_in_4bit=False,  # Load the model with quantized weights
    device_map="auto"   # Automatically distribute the model across available GPUs
)
# Print the number of hidden layers
print(f"Number of hidden layers: {model.config.num_hidden_layers}")
# Freeze all parameters except the last two layers
# Freeze most parameters in the base model but unfreeze the last few layers for fine-tuning
for name, param in model.named_parameters():
    if "decoder.layers" in name and any(str(i) in name for i in range(22, 24)):  # Unfreeze last two layers (example: layers 22 and 23)
        param.requires_grad = True
    else:
        param.requires_grad = False

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your dataset from JSON
df_glaive = pd.read_json("hf://datasets/glaiveai/glaive-code-assistant/c9bc9129-eba0-4b10-8292-4ae70fc7fa0d.json")

# Convert the DataFrame to a HuggingFace Dataset
dataset = Dataset.from_pandas(df_glaive)

# Split the dataset into training and validation sets (90% train, 10% validation)
train_data = dataset.select([i for i in range(len(dataset)) if i % 10 != 0])  # 90% for training
val_data = dataset.select([i for i in range(len(dataset)) if i % 10 == 0])  # 10% for validation

# Tokenize the input and target sequences
def tokenize_function(examples):
    inputs = tokenizer(examples['question'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    labels = tokenizer(examples['answer'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    return {'input_ids': inputs['input_ids'], 'labels': labels['input_ids']}

# Apply tokenization to the datasets
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Print a sample for debugging
print(train_data[0])
print(val_data[0])

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./facebook_last_two_layers_glaive",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    fp16=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)

# Start Training
trainer.train()


Number of hidden layers: 24


Map:   0%|          | 0/122498 [00:00<?, ? examples/s]

Map:   0%|          | 0/13611 [00:00<?, ? examples/s]

{'question': "How can I install Python 3 on an AWS EC2 instance? I tried using the command `sudo yum install python3`, but I received an error message saying `No package python3 available.`. I searched online but didn't find a solution. Do I need to download and install it manually?", 'answer': "To install Python 3 on an AWS EC2 instance, you can use the Amazon Linux Extras Library. This library is a curated set of software that Amazon provides for the Amazon Linux 2 platform. It includes newer versions of software, like Python 3, that are not included in the default Amazon Linux 2 repositories. Here is a step by step process on how to do it:\n\n1. First, update your instance with the following command:\n\n```bash\nsudo yum update -y\n```\n\n2. Next, list available packages in the Amazon Linux Extras repository by typing:\n\n```bash\nsudo amazon-linux-extras list\n```\n\n3. You should see python3.8 available in the list. To install it, use the following command:\n\n```bash\nsudo amazon

Map:   0%|          | 0/122498 [00:00<?, ? examples/s]

...

  0%|          | 0/3403 [00:00<?, ?it/s]

{'eval_loss': 5.162957668304443, 'eval_runtime': 179.5335, 'eval_samples_per_second': 75.813, 'eval_steps_per_second': 18.955, 'epoch': 0.0}
{'loss': 5.1803, 'grad_norm': 20.055217742919922, 'learning_rate': 1.9989551020408163e-05, 'epoch': 0.0}


  0%|          | 0/3403 [00:00<?, ?it/s]

KeyboardInterrupt: 

: 